## Hyperparam Tuning

XGBoost was our strongest performing model in 2 - Model Selection
- Looking into best params for XGBoost
- n_estimators Range: [50, 100, 200, 500]
- Range: [0.01, 0.05, 0.1, 0.2, 0.3]
- A lower learning rate (e.g., 0.01, 0.05) usually requires more n_estimators, but it helps avoid overfitting. learning_rate = 0.1 is commonly used as a baseline.
- max_depth (Maximum Depth of a Tree) Range: [3, 5, 7, 9, 12] Deeper trees can lead to more complex models but may overfit the data. A value between 3 and 7 is typically effective for most datasets, but for more complex data, you can explore depths beyond that.

GridSearchCV can be used to determine these 

There is a fairly significant issue with this approach for this particular problem (described below). But in the interest of creating a basic functional pipeline, you can just use the default Sklearn methods for now.

## Hyperparam Tuning

In [5]:
# perform tuning and cross validation here 
# using GridsearchCV/ RandomsearchCV (MVP)
# or your custom functions

import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

# Load your dataset
# data = pd.read_csv("/Users/spencer.fargey/lighthouselabs/tdsf-midterm/df_linear_model.csv")
data = pd.read_csv("/Users/thomasdoherty/Desktop/tdsf-midterm/df_linear_model.csv")


# Define the features and target
X = data.drop('description.sold_price', axis=1)
y = data['description.sold_price']

# Apply percentile capping as done previously to prevent extreme outliers
lower_percentile = 0.01
upper_percentile = 0.99
lower_bound = y.quantile(lower_percentile)
upper_bound = y.quantile(upper_percentile)
y_capped = y.clip(lower_bound, upper_bound)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_capped, test_size=0.2, random_state=42)

# Create a pipeline with scaling and XGBoost regressor
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Preprocessing step: scaling
    ('xgb', xgb.XGBRegressor(objective='reg:squarederror', random_state=42))  # XGBoost Regressor
])

# Define the hyperparameter grid to search
param_grid = {
    'xgb__n_estimators': [50, 100, 200],   # Number of boosting rounds
    'xgb__learning_rate': [0.01, 0.05, 0.1, 0.2],  # Learning rate (step size shrinkage) - lower learning rates help avoid overfitting
    'xgb__max_depth': [3, 5, 7],  # Maximum depth of a tree
    'xgb__subsample': [0.6, 0.8, 1.0],  # Subsample ratio of the training instances
    'xgb__colsample_bytree': [0.6, 0.8, 1.0],  # Subsample ratio of columns when constructing each tree
    'xgb__alpha': [0, 0.1, 0.5],
    'xgb__lambda': [0.5, 1, 2]
}

# Initialize GridSearchCV with the pipeline and hyperparameters grid
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1)

# Fit the grid search to find the best hyperparameters
grid_search.fit(X_train, y_train)

# Print the best hyperparameters found
print(f"Best hyperparameters: {grid_search.best_params_}")

# Predict on the test data using the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Calculate and print test MSE
mse = mean_squared_error(y_test, y_pred)
print(f"Test MSE: {mse}")

# Optionally, print RMSE for better interpretability
rmse = mse ** 0.5
print(f"Test RMSE: {rmse}")

# Calculate and print R^2 score
r2 = r2_score(y_test, y_pred)
print(f"Test R^2 Score: {r2}")

# Print the best RMSE score from cross-validation
best_rmse = abs(grid_search.best_score_)  # Since 'neg_root_mean_squared_error' returns negative values
print(f"Best cross-validation RMSE: {best_rmse}")


Fitting 5 folds for each of 2916 candidates, totalling 14580 fits
Best hyperparameters: {'xgb__alpha': 0.5, 'xgb__colsample_bytree': 0.6, 'xgb__lambda': 2, 'xgb__learning_rate': 0.1, 'xgb__max_depth': 5, 'xgb__n_estimators': 200, 'xgb__subsample': 0.8}
Test MSE: 50103068496.18618
Test RMSE: 223837.14726601163
Test R^2 Score: 0.5237392538241991
Best cross-validation RMSE: 48260433585.782364


### Thoughts on Results

- R^2 test is still quite average at 52%
- We are supplying data that has already been preprocessed in the 1 - EDA and 2 - model_selection notebooks but we may want to re factor things so that the cleaning and scaling etc we did in 1 - EDA can be done here as part of the pipeline. 
- Currently because we are supplying cleaned data as a file. XGBoost has a cross validation built in but it doesn't do preprocessing so while this works currently, if we wanted to supply new housing data we would need to update the pipeline to do some of the transformations, maybe in a custom function?

We want to make sure that we save our models.  In the old days, one just simply pickled (serialized) the model.  Now, however, certain model types have their own save format.  If the model is from sklearn, it can be pickled, if it's xgboost, for example, the newest format to save it in is JSON, but it can also be pickled.  It's a good idea to stay with the most current methods. 
- you may want to create a new `models/` subdirectory in your repo to stay organized

## Building a Pipeline (Stretch)

> **This step doesn't need to be part of your Minimum Viable Product (MVP), but its highly recommended you complete it if you have time!**

Once you've identified which model works the best, implement a prediction pipeline to make sure that you haven't leaked any data, and that the model could be easily deployed if desired.
- Your pipeline should load the data, process it, load your saved tuned model, and output a set of predictions
- Assume that the new data is in the same JSON format as your original data - you can use your original data to check that the pipeline works correctly
- Beware that a pipeline can only handle functions with fit and transform methods.
- Classes can be used to get around this, but now sklearn has a wrapper for user defined functions.
- You can develop your functions or classes in the notebook here, but once they are working, you should import them from `functions_variables.py` 

In [ ]:
# Build pipeline here

Pipelines come from sklearn.  When a pipeline is pickled, all of the information in the pipeline is stored with it.  For example, if we were deploying a model, and we had fit a scaler on the training data, we would want the same, already fitted scaling object to transform the new data with.  This is all stored when the pipeline is pickled.
- save your final pipeline in your `models/` folder

In [3]:
# save your pipeline here

### Saving our Model

In [7]:

# save your best model here

import joblib

# Save the best model to a file in the models directory
best_model = grid_search.best_estimator_
joblib.dump(best_model, '../models/best_xgboost_model_2_copy.pkl')

print("Best model saved as 'best_xgboost_model_2_copy.pkl'")

Best model saved as 'best_xgboost_model_2_copy.pkl'


In [7]:
# Load the saved model
loaded_model = joblib.load('../models/best_xgboost_model.pkl')

# Use the loaded model for predictions
y_pred_loaded = loaded_model.predict(X_test)



## Post-Project 

## Preventing Data Leakage in Tuning - STRETCH

> **This step doesn't need to be part of your Minimum Viable Product (MVP), but its highly recommended you complete it, if you have time!**

BUT we have a problem - if we calculated a numerical value to encode city (such as the mean of sale prices in that city) on the training data, we can't cross validate 
- The rows in each validation fold were part of the original calculation of the mean for that city - that means we're leaking information!
- While sklearn's built in functions are extremely useful, sometimes it is necessary to do things ourselves

You need to create two functions to replicate what Gridsearch does under the hood. This is a challenging, real world data problem! To help you out, we've created some psuedocode and docstrings to get you started. 

**`custom_cross_validation()`**
- Should take the training data, and divide it into multiple train/validation splits. 
- Look into `sklearn.model_selection.KFold` to accomplish this - the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) shows how to split a dataframe and loop through the indexes of your split data. 
- Within your function, you should compute the city means on the training folds just like you did in Notebook 1 - you may have to re-join the city column to do this - and then join these values to the validation fold

This psuedocode may help you fill in the function:

```python
kfold = KFold() # fit sklearn k folds on X_train
train_folds = []
val_folds = []
for training_index, val_index in kfold.split(X_train):
    train_fold, val_fold = #.iloc loop variables on X_train

    # recompute training city means like you did in notebook 1 
    # merge to validation fold
        
    train_folds.append(train_fold)
    val_folds.append(val_fold)

    return train_folds, val_folds
```


**`hyperparameter_search()`**
- Should take the validation and training splits from your previous function, along with your dictionary of hyperparameter values
- For each set of hyperparameter values, fit your chosen model on each set of training folds, and take the average of your chosen scoring metric. [itertools.product()](https://docs.python.org/3/library/itertools.html) will be helpful for looping through all combinations of hyperparameter values
- Your function should output the hyperparameter values corresponding the highest average score across all folds. Alternatively, it could also output a model object fit on the full training dataset with these parameters.


This psuedocode may help you fill in the function:

```python
hyperparams = # Generate hyperparam options with itertools
hyperparam-scores = []
for hyperparam-combo in hyperparams:

    scores = []

    for folds in allmyfolds:
        # score fold the fold with the model/ hyperparams
        scores.append(score-fold)
        
    score = scores.mean()
    hyperparam-scores.append(score)
# After loop, find max of hyperparam-scores. Best params are at same index in `hyperparams` loop iteratble
```

Docstrings have been provided below to get you started. Once you're done developing your functions, you should move them to `functions_variables.py` to keep your notebook clean 

Bear in mind that these instructions are just one way to tackle this problem - the inputs and output formats don't need to be exactly as specified here.

In [3]:
# develop your custom functions here

def custom_cross_validation(training_data, n_splits =5):
    '''creates n_splits sets of training and validation folds

    Args:
      training_data: the dataframe of features and target to be divided into folds
      n_splits: the number of sets of folds to be created

    Returns:
      A tuple of lists, where the first index is a list of the training folds, 
      and the second the corresponding validation fold

    Example:
        >>> output = custom_cross_validation(train_df, n_splits = 10)
        >>> output[0][0] # The first training fold
        >>> output[1][0] # The first validation fold
        >>> output[0][1] # The second training fold
        >>> output[1][1] # The second validation fold... etc.
    '''

    return training_folds, validation_folds

def hyperparameter_search(training_folds, validation_folds, param_grid):
    '''outputs the best combination of hyperparameter settings in the param grid, 
    given the training and validation folds

    Args:
      training_folds: the list of training fold dataframes
      validation_folds: the list of validation fold dataframes
      param_grid: the dictionary of possible hyperparameter values for the chosen model

    Returns:
      A list of the best hyperparameter settings based on the chosen metric

    Example:
        >>> param_grid = {
          'max_depth': [None, 10, 20, 30],
          'min_samples_split': [2, 5, 10],
          'min_samples_leaf': [1, 2, 4],
          'max_features': ['sqrt', 'log2']} # for random forest
        >>> hyperparameter_search(output[0], output[1], param_grid = param_grid) 
        # assuming 'ouput' is the output of custom_cross_validation()
        [20, 5, 2, 'log2'] # hyperparams in order
    '''

    return hyperparameters